In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_
import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

In [2]:
grid_df = pd.read_pickle('grid_df.pkl')

In [4]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20158410 entries, 0 to 20158409
Data columns (total 61 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   id                         object 
 1   item_id                    int16  
 2   dept_id                    int8   
 3   store_id                   int8   
 4   cat_id                     int8   
 5   state_id                   int8   
 6   d                          int16  
 7   sales                      float16
 8   month                      int8   
 9   event_name_1               int8   
 10  event_type_1               int8   
 11  event_name_2               int8   
 12  event_type_2               int8   
 13  snap_CA                    float16
 14  snap_TX                    float16
 15  snap_WI                    float16
 16  sell_price                 float16
 17  release                    int16  
 18  day                        int8   
 19  week                       int8   
 20  

In [5]:
remove_feature = ['id',
                  'state_id',
                  'store_id',
#                   'item_id',
#                   'dept_id',
#                   'cat_id',
                 'd','sales']

In [31]:
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'max depth': 50
                    'subsample': 0.5,
                    'learning_rate': 0.02,
                    'num_leaves': 100,
                    'min_data_in_leaf': 100,
                    'feature_fraction': 0.5,
                    'n_estimators': 500,
                    'verbose': -1,
                    'seed' : 42
                }

In [10]:
stores = list(grid_df['store_id'].unique())

In [18]:
validation = {'public' : [1913, 1941], 'private' : [1941, 1969] }

## Store Level Model

In [34]:
for store in stores:
    df = grid_df[grid_df['store_id'] == store]
    inputCols = df.columns[~df.columns.isin(remove_feature)]
    train = (df['d'] <= validation['private'][0]) & (df['d'] >= FIRST_DAY)
    val = (df['d'] > validation['private'][0]) & (df['d'] <= validation['private'][1])

    train_data = lgb.Dataset(df[train][inputCols], label = df[train]['sales'])

    valid_data = lgb.Dataset(df[val][inputCols], label = df[val]['sales'])
    
    m_lgb = lgb.train(lgb_params, train_data, valid_sets = [valid_data, train_data], verbose_eval=100) 
    display(pd.DataFrame({'name':m_lgb.feature_name(),
                              'imp':m_lgb.feature_importance()}).sort_values('imp',ascending=False).head(10))

    model_name = "StoreModels/" + str(store)+'.bin'
    pickle.dump(m_lgb, open(model_name, 'wb'))
    
    indice = df[val].index.tolist()
    prediction = pd.DataFrame({'y_pred' : m_lgb.predict(df[val][inputCols])})
    prediction.index = indice
    
    grid_1 = pd.read_pickle("df_final_1.pkl")
    pd.concat([grid_1.iloc[indice], prediction], axis=1)\
        .pivot(index='id', columns='d', values='y_pred')\
        .reset_index()\
        .set_index('id')\
        .to_csv("StorePredictions/"+"store"+ str(store)+'.csv')
        
    del df, train_data, valid_data, m_lgb, train, val; gc.collect, grid_1
    

0 start
[100]	training's rmse: 2.47496	valid_0's rmse: 2.41058
[200]	training's rmse: 2.2484	valid_0's rmse: 3.06623
[300]	training's rmse: 2.17891	valid_0's rmse: 3.24316
[400]	training's rmse: 2.13435	valid_0's rmse: 3.30275
[500]	training's rmse: 2.09936	valid_0's rmse: 3.32686
[600]	training's rmse: 2.07137	valid_0's rmse: 3.33739
[700]	training's rmse: 2.04667	valid_0's rmse: 3.3362
[800]	training's rmse: 2.02459	valid_0's rmse: 3.34015
[900]	training's rmse: 2.0043	valid_0's rmse: 3.34499
[1000]	training's rmse: 1.98601	valid_0's rmse: 3.34527


,name,imp
14,week,12319
0,item_id,10535
54,rolling_mean_180,9191
29,enc_item_id_store_id_mean,9013
30,enc_item_id_store_id_std,8629
55,rolling_std_180,8180
28,enc_item_id_state_id_std,7979
11,sell_price,7902
52,rolling_mean_60,7758
53,rolling_std_60,7416


1 start
[100]	training's rmse: 2.00881	valid_0's rmse: 2.14484
[200]	training's rmse: 1.85579	valid_0's rmse: 2.68574
[300]	training's rmse: 1.80834	valid_0's rmse: 2.85664
[400]	training's rmse: 1.7767	valid_0's rmse: 2.91492
[500]	training's rmse: 1.75251	valid_0's rmse: 2.93541
[600]	training's rmse: 1.73306	valid_0's rmse: 2.93964
[700]	training's rmse: 1.71621	valid_0's rmse: 2.9399
[800]	training's rmse: 1.70065	valid_0's rmse: 2.939
[900]	training's rmse: 1.68611	valid_0's rmse: 2.93849
[1000]	training's rmse: 1.67203	valid_0's rmse: 2.93899


,name,imp
14,week,12349
0,item_id,10976
29,enc_item_id_store_id_mean,9812
54,rolling_mean_180,9743
30,enc_item_id_store_id_std,9418
55,rolling_std_180,8985
52,rolling_mean_60,8325
28,enc_item_id_state_id_std,8057
53,rolling_std_60,7595
11,sell_price,7593


2 start
[100]	training's rmse: 3.38268	valid_0's rmse: 3.49077
[200]	training's rmse: 2.97301	valid_0's rmse: 4.54848
[300]	training's rmse: 2.86574	valid_0's rmse: 4.8326
[400]	training's rmse: 2.79608	valid_0's rmse: 4.91548
[500]	training's rmse: 2.74314	valid_0's rmse: 4.90872
[600]	training's rmse: 2.70023	valid_0's rmse: 4.91921
[700]	training's rmse: 2.66535	valid_0's rmse: 4.92011
[800]	training's rmse: 2.63294	valid_0's rmse: 4.91898
[900]	training's rmse: 2.60043	valid_0's rmse: 4.93287
[1000]	training's rmse: 2.57182	valid_0's rmse: 4.92894


...

,name,imp
14,week,13193
0,item_id,10505
54,rolling_mean_180,9446
29,enc_item_id_store_id_mean,9255
30,enc_item_id_store_id_std,9161
28,enc_item_id_state_id_std,8524
55,rolling_std_180,8366
52,rolling_mean_60,8127
11,sell_price,8080
12,release,7906


8 start
[100]	training's rmse: 2.8703	valid_0's rmse: 3.00644
[200]	training's rmse: 2.63583	valid_0's rmse: 3.85537
[300]	training's rmse: 2.55103	valid_0's rmse: 4.07791
[400]	training's rmse: 2.48307	valid_0's rmse: 4.14886
[500]	training's rmse: 2.43038	valid_0's rmse: 4.15549
[600]	training's rmse: 2.38644	valid_0's rmse: 4.15625
[700]	training's rmse: 2.34943	valid_0's rmse: 4.1512
[800]	training's rmse: 2.3154	valid_0's rmse: 4.14176
[900]	training's rmse: 2.28316	valid_0's rmse: 4.14489
[1000]	training's rmse: 2.25396	valid_0's rmse: 4.13243


,name,imp
14,week,11831
0,item_id,10384
54,rolling_mean_180,8961
29,enc_item_id_store_id_mean,8257
30,enc_item_id_store_id_std,7983
11,sell_price,7666
52,rolling_mean_60,7648
55,rolling_std_180,7586
28,enc_item_id_state_id_std,7536
12,release,7425


9 start
[100]	training's rmse: 2.21216	valid_0's rmse: 2.42189
[200]	training's rmse: 1.97069	valid_0's rmse: 3.09109
[300]	training's rmse: 1.89192	valid_0's rmse: 3.27719
[400]	training's rmse: 1.84062	valid_0's rmse: 3.34207
[500]	training's rmse: 1.80351	valid_0's rmse: 3.36003
[600]	training's rmse: 1.77426	valid_0's rmse: 3.36487
[700]	training's rmse: 1.74811	valid_0's rmse: 3.3668
[800]	training's rmse: 1.72513	valid_0's rmse: 3.35718
[900]	training's rmse: 1.70379	valid_0's rmse: 3.35533
[1000]	training's rmse: 1.68424	valid_0's rmse: 3.34865


,name,imp
14,week,12309
0,item_id,11169
54,rolling_mean_180,9370
30,enc_item_id_store_id_std,8615
29,enc_item_id_store_id_mean,8607
55,rolling_std_180,8422
11,sell_price,8071
52,rolling_mean_60,8043
28,enc_item_id_state_id_std,8016
53,rolling_std_60,7272


In [36]:
for file in os.listdir("StorePredictions/"):
    print(file)

submission_onlystore0.csv
submission_onlystore1.csv
submission_onlystore2.csv
submission_onlystore3.csv
submission_onlystore4.csv
submission_onlystore5.csv
submission_onlystore6.csv
submission_onlystore7.csv
submission_onlystore8.csv
submission_onlystore9.csv


In [39]:
submission = pd.read_csv('sample_submission.csv').set_index('id').iloc[30490:]
sub_id = pd.DataFrame({'id':submission.index.tolist()})
fcol = [f'F{i}' for i in range(1,29)]
sub_copy = submission.copy()

In [41]:
for file in os.listdir("StorePredictions/"):
    temp = pd.read_csv("StorePredictions/"+file)
    temp.columns = ['id']+fcol
    sub_copy += sub_id.merge(temp, how='left', on='id').set_index('id').fillna(0)
    sub_copy.columns = fcol
    sub_copy.to_csv("StorePredictions/"+'store.csv')

##